In [ ]:
import os

from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, transform
import torch
from torchvision.utils import make_grid
from tqdm import tqdm
prefix=os.path.abspath(os.path.dirname(os.getcwd()))+"/"


In [ ]:
breed = prefix+"data/breed.csv"
filename = prefix+"data/labels.csv"
labelsNames = ["id", "breed"]
labels = read_csv(filename, names=labelsNames)

breedNames = read_csv(breed)["breed"].tolist()[1:]
print(labels.shape)
# print(len(breedNames))

In [ ]:
le = LabelEncoder()
le.fit(list(set(labels["breed"].tolist()[1:])))
data_label = le.transform(labels["breed"].tolist()[1:])
print(len(data_label))

In [ ]:
data_transforms = {
    "train": transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize(256),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(),  # Apply random horizontal flip
            transforms.ToTensor(),
            transforms.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]  # Normalize tensor image with mean and std
            ),  # input = (input - mean) / std
        ]
    ),
    "val": transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize(256),
            transforms.CenterCrop(224),  # For validation, only center crop is needed
            transforms.ToTensor(),
            transforms.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]  # Normalize tensor image with mean
            ),  # and std
        ]
    ),
}

In [ ]:
from sklearn.model_selection import train_test_split

labels_files = labels["id"].apply(lambda x: prefix+"/data/train/" + x + ".jpg").tolist()[1:]
train_path, val_path, train_label, val_label = train_test_split(
    labels_files, data_label, test_size=0.2
)


In [ ]:
from torch.utils.data import Dataset
from skimage import io  # Assuming you are using skimage to read images

class DogDataset(Dataset):
    def __init__(self, data_path, data_label, transform=None):
        """
        - data_path (string): Path to the images
        - data_label (string): Labels for the images
        - transform (callable, optional): A function/transform to apply to each sample
        """
        self.data_path = data_path
        self.data_label = data_label
        self.transform = transform

    def __len__(self):
        return len(self.data_path)

    def __getitem__(self, idx):
        img_path = self.data_path[idx]
        image = io.imread(img_path)
        label = self.data_label[idx]

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
train_dataset = DogDataset(train_path, train_label, data_transforms["train"])
val_dataset = DogDataset(val_path, val_label, data_transforms["val"])


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)


In [ ]:

for batch_index, sample_batch in enumerate(train_loader):
    images, labels = sample_batch
    sample_images = make_grid(images, normalize=True)
    plt.figure(figsize=(8, 8))
    plt.imshow(sample_images.permute(1, 2, 0).numpy())
    break

In [ ]:
from torchvision import models

alexnet = models.alexnet(pretrained=True)
alexnet

In [ ]:
for param in alexnet.parameters():
    param.requires_grad = False
    print(param.requires_grad)

In [ ]:
classifier = list(alexnet.classifier.children())  # Get all layers of the classifier
# Replace the last layer from Linear(4096, 1000) to Linear(4096, 120)
classifier[-1] = torch.nn.Linear(4096, 120)
classifier.append(torch.nn.Softmax(dim=1))  # Add a Softmax layer
alexnet.classifier = torch.nn.Sequential(*classifier)  # Update the original classifier
alexnet

In [ ]:
dev = torch.device("cuda") 

In [ ]:
criterion = torch.nn.CrossEntropyLoss()  # Cross-entropy loss function
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, alexnet.parameters()), lr=0.001
)  # Optimizer
# Learning rate decay: after every 1 iteration, decay learning rate by a factor of 0.5
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

criterion, optimizer, lr_scheduler

In [1]:
epochs = 100
model = alexnet.to(dev)
print("Start Training...")
if os.path.exists(prefix + "models/model.pt"):  # Check if a pre-trained model exists
    model_saved = alexnet
    model_saved.load_state_dict(torch.load(prefix+"models/model.pt", weights_only=True))
    model = model_saved.to(dev)
    print("Load model from model.pt")

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(dev)  # Move images to the device (GPU or CPU)
        labels = labels.to(dev)  # Move labels to the device (GPU or CPU)

        outputs = model(images)
        loss = criterion(outputs, labels.type(torch.int64))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(
                "Epoch [{}/{}], Batch [{}/{}], Train loss: {:.3f}".format(
                    epoch + 1, epochs, i + 1, len(train_loader), loss.item()
                )
            )

    correct = 0
    total = 0
    if epoch % 10 == 0:
        print("Save model to model.pt")
        torch.save(model.state_dict(), prefix + "models/model.pt")

    for images, labels in val_loader:
        images = images.to(dev)  # Move images to the device (GPU or CPU)
        labels = labels.to(dev)  # Move labels to the device (GPU or CPU)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    print("============ Test accuracy: {:.3f} =============".format(correct / total))

    lr_scheduler.step()  # Apply learning rate decay

NameError: name 'alexnet' is not defined

In [ ]:
torch.save(model.state_dict(), prefix+"models/model.pt")

In [ ]:
IMAGE = io.imread("data/train/0AjbbdBwNiGqWo5n.jpg")
IMAGE = data_transforms["val"](IMAGE).unsqueeze(0)  
IMAGE.size()

In [ ]:
data= model_saved(IMAGE.to(dev))
# probality = model_saved(IMAGE.to(dev))
# print(probality)
# print(le.inverse_transform([data]))
tempDict = {}
for i in range(len(data[0])):
    tempDict[le.inverse_transform([i])[0]] = data[0][i].item()
temp=pd.DataFrame(tempDict,index=[0])
temp.insert(0,"id","0AjbbdBwNiGqWo5n")
temp

In [ ]:
outdata= pd.DataFrame(columns=["id"]+breedNames)
outdata
testPath = os.getcwd()+"/data/test/"
files= os.listdir(testPath)

p=0

for file in tqdm(files):
    p+=1
    IMAGE = io.imread(testPath+file)
    IMAGE = data_transforms["val"](IMAGE).unsqueeze(0)
    data= model_saved(IMAGE.to(dev))
    tempDict = {}
    for i in range(len(data[0])):
        tempDict[le.inverse_transform([i])[0]] = data[0][i].item()
    temp=pd.DataFrame(tempDict,index=[0])
    temp.insert(0,"id",file[:-4])
    outdata=pd.concat([outdata,temp],ignore_index=True)
outdata

In [ ]:
output_path = "output.csv" 
columns_sorted = ["id"] + sorted(col for col in outdata.columns if col != "id")
outdata = outdata[columns_sorted]
outdata.to_csv(output_path, index=False)  